# Analysis: 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [6]:
movies = pd.read_csv('movies.csv')
movies.head()

,year,rank,release,gross,adjusted_gross,theaters,total_gross,adjusted_total_gross,release_date,distributor,release_date_formatted,seasons
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,203359628,782934567,1278,209398025,806182396,May 21,Twentieth Century Fox,1980-05-21,summer
1,1980,2,9 to 5,103290500,397668425,910,103290500,397668425,Dec 19,Twentieth Century Fox,1980-12-19,holiday season
2,1980,3,Stir Crazy,101300000,390005000,813,101300000,390005000,Dec 12,Columbia Pictures,1980-12-12,holiday season
3,1980,4,Kramer vs. Kramer,94448310,363625993,898,106260000,409101000,Dec 19,Columbia Pictures,1980-12-19,holiday season
4,1980,5,Any Which Way You Can,70687344,272146274,1541,70687344,272146274,Dec 17,Warner Bros.,1980-12-17,holiday season


### viz 1: totals for each season -- idk bar graph or size comparison

In [7]:
movies_grouped = movies.groupby('seasons')
movies_grouped_seasons = movies_grouped['seasons'].value_counts()

movies_grouped_seasons.sort_values(ascending=False)

seasons
summer            1378
holiday season    1097
spring             634
fall               612
winter             502
Name: count, dtype: int64

seasons, total_count
summer, 1378
holiday season, 1097
spring, 635
fall, 612
winter, 502

### viz 2: seasons heatmap

In [8]:
movies_1980 = movies[movies['year'] == 1980].groupby('seasons')
movies_1980_seasons = movies_1980['seasons'].value_counts()

movies_1980_season_counts = movies_1980_seasons.sort_values(ascending=False)

print(movies_1980_season_counts)

seasons
summer            25
fall              13
holiday season    13
winter             9
spring             8
Name: count, dtype: int64


In [9]:
# rerun to reset df

header = ['year', 'summer', 'fall', 'spring', 'winter', 'holiday season']

df = pd.DataFrame(columns=header)

static test with 2018 -- SKIP

In [115]:
year = 1980

df.loc[0] = [year, movies_1980_season_counts['summer'], movies_1980_season_counts['fall'], movies_1980_season_counts['spring'], movies_1980_season_counts['winter'], movies_1980_season_counts['holiday season']]

df.astype(int)

,year,summer,fall,spring,winter,holiday season
0,1980,25,13,8,9,13


let's try to loop this

In [10]:
years = range(1980, 2023)

for year in years:
    movies_year = movies[movies['year'] == year].groupby('seasons')

    movies_year_seasons = movies_year['seasons'].value_counts()

    movies_year_season_counts = movies_year_seasons.sort_values(ascending=False)

    df.loc[len(df)] = [year, movies_year_season_counts.get('summer', 0), movies_year_season_counts.get('fall', 0), movies_year_season_counts.get('spring', 0), movies_year_season_counts.get('winter', 0), movies_year_season_counts.get('holiday season', 0)]

df

,year,summer,fall,spring,winter,holiday season
0,1980,25,13,8,9,13
1,1981,21,6,13,2,14
2,1982,41,8,13,11,27
3,1983,35,14,17,8,26
4,1984,28,16,17,10,29
5,1985,35,14,16,9,26
6,1986,38,12,13,13,24
7,1987,36,15,13,11,25
8,1988,34,9,19,11,27
9,1989,31,13,21,10,25


In [117]:
# random checks to verify accuracy

len(movies[(movies['year'] == 2020) & (movies['seasons'] == 'summer')])

17

now let's melt this tidy data style B)

In [11]:
tidy_df = df.melt(id_vars='year', var_name='season', value_name='count')
tidy_df[tidy_df['year'] == 1980]

,year,season,count
0,1980,summer,25
43,1980,fall,13
86,1980,spring,8
129,1980,winter,9
172,1980,holiday season,13


some more analysis: years where summer is not the most popular season

In [12]:
top_season = df[['summer', 'fall', 'spring', 'winter', 'holiday season']].idxmax(axis=1)

df['top_season'] = top_season

df

,year,summer,fall,spring,winter,holiday season,top_season
0,1980,25,13,8,9,13,summer
1,1981,21,6,13,2,14,summer
2,1982,41,8,13,11,27,summer
3,1983,35,14,17,8,26,summer
4,1984,28,16,17,10,29,holiday season
5,1985,35,14,16,9,26,summer
6,1986,38,12,13,13,24,summer
7,1987,36,15,13,11,25,summer
8,1988,34,9,19,11,27,summer
9,1989,31,13,21,10,25,summer


### 

In [16]:
df.to_csv('seasons_totals.csv', index=False)
tidy_df.to_csv('seasons_totals_tidy.csv', index=False)

### viz 3: distributors preferences

In [13]:
season_counts = movies.dropna(subset=['seasons']).groupby('distributor')['seasons'].value_counts()

distributors_df = pd.DataFrame(season_counts)
distributors_df

count
distributor                       seasons              
-                                 spring             18
                                  holiday season     17
                                  summer             15
                                  fall               10
                                  winter              5
...                                                 ...
Warner Independent Pictures (WIP) summer              1
Well Go USA Entertainment         summer              2
                                  holiday season      1
Yari Film Group Releasing         summer              1
truTV                             winter              1

[279 rows x 1 columns]

top 10 distributors



In [14]:
top_distributors = movies['distributor'].value_counts().head(10)
top_distributors_list = top_distributors.index.to_list()
top_distributors_list

['Warner Bros.',
 'Universal Pictures',
 'Walt Disney Studios Motion Pictures',
 'Twentieth Century Fox',
 'Paramount Pictures',
 'Sony Pictures Entertainment (SPE)',
 'Metro-Goldwyn-Mayer (MGM)',
 'Lionsgate',
 'Columbia Pictures',
 'New Line Cinema']

In [15]:
distributors = movies[movies['distributor'].isin(top_distributors_list)]
distributors

,year,rank,release,gross,adjusted_gross,theaters,total_gross,adjusted_total_gross,release_date,distributor,release_date_formatted,seasons
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,203359628,782934567,1278,209398025,806182396,May 21,Twentieth Century Fox,1980-05-21,summer
1,1980,2,9 to 5,103290500,397668425,910,103290500,397668425,Dec 19,Twentieth Century Fox,1980-12-19,holiday season
2,1980,3,Stir Crazy,101300000,390005000,813,101300000,390005000,Dec 12,Columbia Pictures,1980-12-12,holiday season
3,1980,4,Kramer vs. Kramer,94448310,363625993,898,106260000,409101000,Dec 19,Columbia Pictures,1980-12-19,holiday season
4,1980,5,Any Which Way You Can,70687344,272146274,1541,70687344,272146274,Dec 17,Warner Bros.,1980-12-17,holiday season
...,...,...,...,...,...,...,...,...,...,...,...,...
4212,2022,89,Fall,7240521,7674952,1548,7240521,7674952,Aug 12,Lionsgate,2022-08-12,summer
4214,2022,91,Ghostbusters: Afterlife,6981615,7400511,4315,129360575,137122209,Nov 19,Sony Pictures Entertainment (SPE),2022-11-19,holiday season
4217,2022,94,She Said,5825995,6175554,2023,5825995,6175554,Nov 18,Universal Pictures,2022-11-18,holiday season
4220,2022,97,Encanto,5465293,5793210,3980,96093622,101859239,Nov 24,Walt Disney Studios Motion Pictures,2022-11-24,holiday season


In [16]:
distributors_seasons = distributors.groupby('distributor')['seasons'].value_counts(ascending=False)

# TO RESET DF

In [17]:
distributors_seasons_df = pd.DataFrame(distributors_seasons)
distributors_seasons_df = distributors_seasons_df.reset_index()

distributors_seasons_df

,distributor,seasons,count
0,Columbia Pictures,summer,41
1,Columbia Pictures,holiday season,39
2,Columbia Pictures,fall,23
3,Columbia Pictures,spring,17
4,Columbia Pictures,winter,9
5,Lionsgate,summer,32
6,Lionsgate,holiday season,28
7,Lionsgate,fall,26
8,Lionsgate,spring,25
9,Lionsgate,winter,20


### NVM : who's coming out on top?
average gross for each season

In [38]:
movies_test = movies[(movies['distributor'] == 'Twentieth Century Fox') & (movies['seasons'] == 'summer')]
movies_test_gross = movies_test['adjusted_gross'].max()

movies_test_gross

783113108

In [18]:
seasons = ['summer', 'fall', 'spring', 'winter', 'holiday season']

for distributor in top_distributors_list:
    for season in seasons:
        movies_filtered = movies[(movies['distributor'] == distributor) & (movies['seasons'] == season)]
        movies_gross = movies_filtered['adjusted_gross'].mean()
        distributors_seasons_df.loc[
            (distributors_seasons_df['distributor'] == distributor) & (distributors_seasons_df['seasons'] == season),
            'average_adjusted_gross'
        ] = round(movies_gross, 0)

distributors_seasons_df['average_adjusted_gross'] = distributors_seasons_df['average_adjusted_gross'].astype(int)
distributors_seasons_df

,distributor,seasons,count,average_adjusted_gross
0,Columbia Pictures,summer,41,125098311
1,Columbia Pictures,holiday season,39,94398449
2,Columbia Pictures,fall,23,58599908
3,Columbia Pictures,spring,17,44459054
4,Columbia Pictures,winter,9,49592802
5,Lionsgate,summer,32,71078150
6,Lionsgate,holiday season,28,104033627
7,Lionsgate,fall,26,67098261
8,Lionsgate,spring,25,85382735
9,Lionsgate,winter,20,63013881


top + least grossing film of each season for each distributor

In [46]:
# disney_movies = movies[movies['distributor'] == 'Walt Disney Studios Motion Pictures']
# disney_movies_seasons = disney_movies.groupby('seasons')

# # find index of the row with the highest 'gross' value for each season
# disney_top_grossing_index = disney_movies_seasons['adjusted_gross'].idxmax()

# disney_top_grossing_movies = disney_movies.loc[disney_top_grossing_index]

# disney_top_grossing_movies[['seasons', 'adjusted_gross', 'release', 'year']]

,seasons,adjusted_gross,release,year
2026,fall,406067277,"Monsters, Inc.",2001
3425,holiday season,834518104,Star Wars: Episode VII - The Force Awakens,2015
3824,spring,1021463870,Avengers: Endgame,2019
3726,summer,736383910,Incredibles 2,2018
3724,winter,847072074,Black Panther,2018


In [184]:
# movies[(movies['distributor'] == 'Walt Disney Studios Motion Pictures') & (movies['seasons'] == 'summer')].sort_values(by='gross', ascending = False)

### top film for each season + year released

In [51]:
# distributors_seasons_df['top_release'] = pd.NA
# distributors_seasons_df['top_year'] = pd.NA
# distributors_seasons_df['top_gross'] = pd.NA

# for distributor in top_distributors_list:
#     for season in seasons:
#         distributor_movies = movies[(movies['distributor'] == distributor) & (movies['seasons'] == season)]

#         if not distributor_movies.empty:
#             top_grossing_index = distributor_movies['adjusted_gross'].idxmax()
#             top_grossing_release = distributor_movies.loc[top_grossing_index, 'release']
#             top_grossing_year = distributor_movies.loc[top_grossing_index, 'year']
#             top_grossing_gross = distributor_movies.loc[top_grossing_index, 'adjusted_gross']
            

#             distributors_seasons_df.loc[(distributors_seasons_df['distributor'] == distributor) &
#                                         (distributors_seasons_df['seasons'] == season),
#                                         'top_release'] = top_grossing_release

#             distributors_seasons_df.loc[(distributors_seasons_df['distributor'] == distributor) &
#                                         (distributors_seasons_df['seasons'] == season),
#                                         'top_year'] = top_grossing_year
            
#             distributors_seasons_df.loc[(distributors_seasons_df['distributor'] == distributor) &
#                                         (distributors_seasons_df['seasons'] == season),
#                                         'top_gross'] = top_grossing_gross
                                

# distributors_seasons_df

,distributor,seasons,count,average_adjusted_gross,top_release,top_year,top_gross
0,Columbia Pictures,summer,41,125098311,Ghostbusters,1984,649504791
1,Columbia Pictures,holiday season,39,94398449,Tootsie,1983,447553522
2,Columbia Pictures,fall,23,58599908,Chariots of Fire,1982,186608050
3,Columbia Pictures,spring,17,44459054,Richard Pryor: Live on the Sunset Strip,1982,115070112
4,Columbia Pictures,winter,9,49592802,Groundhog Day,1993,148904643
5,Lionsgate,summer,32,71078150,John Wick: Chapter 3 - Parabellum,2019,203508667
6,Lionsgate,holiday season,28,104033627,The Hunger Games: Catching Fire,2013,514184716
7,Lionsgate,fall,26,67098261,Saw II,2005,136584087
8,Lionsgate,spring,25,85382735,The Hunger Games,2012,538574113
9,Lionsgate,winter,20,63013881,Madea Goes to Jail,2009,128521837


# actually let's redo that

In [21]:
movies_gross = pd.read_csv('movies_gross.csv')
movies_gross.head()


,release,distributor,seasons,adjusted_gross
0,'83,Reliance Entertainment,holiday season,3448423
1,*batteries not included,Universal Pictures,holiday season,86795885
2,10 Cloverfield Lane,Paramount Pictures,spring,90824577
3,10 Things I Hate About You,Walt Disney Studios Motion Pictures,spring,69484262
4,10 to Midnight,Metro-Goldwyn-Mayer (MGM),spring,21957311


In [22]:
distributors_seasons_df['top_release'] = pd.NA
distributors_seasons_df['top_gross'] = pd.NA

for distributor in top_distributors_list:
    for season in seasons:
        distributor_movies = movies_gross[(movies_gross['distributor'] == distributor) & (movies_gross['seasons'] == season)]

        if not distributor_movies.empty:
            top_grossing_index = distributor_movies['adjusted_gross'].idxmax()
            top_grossing_release = distributor_movies.loc[top_grossing_index, 'release']
            top_grossing_gross = distributor_movies.loc[top_grossing_index, 'adjusted_gross']
            

            distributors_seasons_df.loc[(distributors_seasons_df['distributor'] == distributor) &
                                        (distributors_seasons_df['seasons'] == season),
                                        'top_release'] = top_grossing_release
            
            distributors_seasons_df.loc[(distributors_seasons_df['distributor'] == distributor) &
                                        (distributors_seasons_df['seasons'] == season),
                                        'top_gross'] = top_grossing_gross
                                

distributors_seasons_df

,distributor,seasons,count,average_adjusted_gross,top_release,top_gross
0,Columbia Pictures,summer,41,125098311,Ghostbusters,649504791
1,Columbia Pictures,holiday season,39,94398449,Tootsie,545635464
2,Columbia Pictures,fall,23,58599908,Chariots of Fire,186972727
3,Columbia Pictures,spring,17,44459054,Richard Pryor: Live on the Sunset Strip,115070112
4,Columbia Pictures,winter,9,49592802,Groundhog Day,148904643
5,Lionsgate,summer,32,71078150,John Wick: Chapter 3 - Parabellum,203508667
6,Lionsgate,holiday season,28,104033627,The Hunger Games: Catching Fire,551485633
7,Lionsgate,fall,26,67098261,Saw II,136584087
8,Lionsgate,spring,25,85382735,The Hunger Games,538574113
9,Lionsgate,winter,20,63013881,Madea Goes to Jail,128521837


In [23]:
distributors_seasons_df.to_csv('distributors_seasons.csv', index=False)

In [26]:
movies[(movies['distributor'] == 'Walt Disney Studios Motion Pictures') & (movies['seasons'] == 'summer')].sort_values(by='adjusted_gross', ascending = False)

,year,rank,release,gross,adjusted_gross,theaters,total_gross,adjusted_total_gross,release_date,distributor,release_date_formatted,seasons
3726,2018,3,Incredibles 2,608581744,736383910,4410,608581744,736383910,Jun 15,Walt Disney Studios Motion Pictures,2018-06-15,summer
3825,2019,2,The Lion King,543638043,646929271,4802,543638043,646929271,Jul 19,Walt Disney Studios Motion Pictures,2019-07-19,summer
2524,2006,1,Pirates of the Caribbean: Dead Man's Chest,423317154,639208902,4133,423315812,639206876,Jul 7,Walt Disney Studios Motion Pictures,2006-07-07,summer
3524,2016,1,Finding Dory,486295561,612732406,4305,486295561,612732406,Jun 17,Walt Disney Studios Motion Pictures,2016-06-17,summer
1324,1994,1,The Lion King,295691076,606166705,2624,312855561,641353900,Jun 15,Walt Disney Studios Motion Pictures,1994-06-15,summer
...,...,...,...,...,...,...,...,...,...,...,...,...
504,1985,81,Return to Oz,11137801,31631354,1238,11137801,31631354,Jun 21,Walt Disney Studios Motion Pictures,1985-06-21,summer
1520,1995,97,Mad Love,15453274,30752015,1500,15453274,30752015,May 26,Walt Disney Studios Motion Pictures,1995-05-26,summer
1219,1992,96,A Stranger Among Us,12282994,26654096,684,12282994,26654096,Jul 17,Walt Disney Studios Motion Pictures,1992-07-17,summer
920,1989,97,Cheetah,8153677,20139582,1321,8153677,20139582,Aug 18,Walt Disney Studios Motion Pictures,1989-08-18,summer
